In [ ]:
import requests
import json
import base64
from datetime import datetime
import csv

CREATION_TIME = datetime.now().strftime("%Y-%m-%d-%H%M%S")
# export it to CSV file
def write_data_to_csv(data, name):
    fileName = '{}{}.csv'.format(name,CREATION_TIME )
    with open(fileName, 'w', newline='' , encoding="utf-8") as csvfile:
        print("Writing {}".format(fileName))
        fieldnames = list(data[1].keys())
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        for v in data:
            writer.writerow(v)



def extract_data_from_api(url, username, password, MaxFileVolume = "999999", startAt = "0"):
    headers = {
        "Authorization": f"Basic {base64.b64encode(f'{username}:{password}'.encode()).decode()}",
        "Content-Type": "application/json"
    }

    all_data = []
    has_more = True
    offset = int(startAt)
    partialExport = int(MaxFileVolume)
    limit = 500

    while has_more:
        params = {
            "offset": offset,
            "limit": limit
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            items = data["items"]
            all_data.extend(items)

            has_more = data["hasMore"]
            offset += limit
        else:
            print(f"Request failed with status code {response.status_code}")
            all_data.extend("Request failed with status code {response.status_code}")
            return all_data
            break
        print("{} rows processed".format(str(offset-int(startAt))))
        if offset - int(startAt) >= partialExport:
            write_data_to_csv(all_data, "PartialyResults")
            partialExport = partialExport + int(MaxFileVolume)



    return all_data
        
# Example usage

# Read API credentials from a text file
def read_credentials_from_file(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()
        api = lines[0].strip()
        username = lines[1].strip()
        password =  lines[2].strip()
        MaxFileVolume = lines[3].strip()
        startAt =  lines[4].strip()
    return api, username, password, MaxFileVolume, startAt

# Example usage
credentials_file = "APIData.txt"  # Replace with the path to your credentials file

# Read API credentials from file
api_url, api_username, api_password, MaxFileVolume, startAt= read_credentials_from_file(credentials_file)

data = extract_data_from_api(api_url, api_username, api_password, MaxFileVolume, startAt)

write_data_to_csv(data, "CompleteResults")

